## Examples of usage

In [1]:
import numpy as np
import random
import pandas as pd
import meliora.src.meliora.core as vt
from scipy.stats import norm

In [2]:
# Import data
test_data_main = pd.read_csv('../data/pd_test_data_main.csv')
test_data_add = pd.read_csv('../data/pd_test_data_period_2.csv')
pd_test_data = pd.read_csv('../data/pd_test_data_main.csv')
test_pd_psi = pd.read_csv('../data/pd_bellini.csv')
german_data = pd.read_csv('../data/german_data.csv')
lgd_t_test = pd.read_csv('../data/lgd_t_test.csv')
pd_transition_matrix = pd.read_csv('../data/pd_transition_matrix.csv')
pd_calibration = pd.read_csv('../data/pd_calibration.csv')
lgd_dataset = pd.read_csv('../data/lgd_dataset.csv')

In [3]:
lgd_t_test.head()

,facility,predicted_lgd,realised_lgd,segment,ead
0,1,0.80,0.83,8,5860
1,2,0.75,0.81,7,6653
2,3,0.42,0.45,4,3192
3,4,0.99,1.00,9,7322
4,5,0.90,0.86,9,5513


In [4]:
lgd_dataset.head()

,realised_outcome,predicted_outcome
0,1,3
1,1,3
2,1,3
3,2,2
4,2,2


In [5]:
pd_transition_matrix.head()

,period_1_ratings,period_2_ratings
0,6,4
1,6,5
2,3,2
3,5,2
4,7,8


In [6]:
# Binomial test
vt.binomial_test(test_data_main, "ratings", "default_flag", "predicted_pd")

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.023892,True
1,B,0.099524,480,41,0.085417,0.867441,False
2,C,0.095719,114,10,0.087719,0.660553,False


In [7]:
# Jeffrey's test
vt.jeffreys_test(test_data_main, 'ratings', 'default_flag', 'predicted_pd', 0.05)

,Rating class,Predicted PD,Total count,Defaults,Actual Default Rate,p_value,Reject H0
0,A,0.103429,406,55,0.135468,0.019959,True
1,B,0.099524,480,41,0.085417,0.849552,False
2,C,0.095719,114,10,0.087719,0.598649,False


In [8]:
# Brier score
vt.brier_score(test_data_main, "ratings", "default_flag", "predicted_pd")

0.0012895084997917349

In [9]:
# Hosmer-Lemeshow test
vt.hosmer_test(test_data_main, "ratings", "default_flag", "predicted_pd")

[0.1302500370047408, False]

In [10]:
# Spiegelhalter test
vt.spiegelhalter_test(test_data_main, "ratings", "default_flag", "predicted_pd")

(-0.6637590511485174, False)

In [11]:
# ROC test
vt.roc_auc(test_data_main, "default_flag", "predicted_pd")

0.5008547549702419

In [12]:
# Spearman correlation
x = [1,2,3,4,5]
y = [5,6,7,8,7]
vt.spearman_correlation(x, y).correlation

0.8207826816681233

In [13]:
# Somers D
x = [0, 1, 1, 1, 1] 
y = [1, 1, 1, 0, 1]
vt.somersd(x, y)

SomersDResult(statistic=-0.25, pvalue=0.3613104285261787, table=array([[0, 1],
       [1, 3]]))

In [14]:
# Kendall tau
x = [1, 2, 3, 2, 1, 3, 4, 2, 5, 2, 6, 5, 5]
y = [5, 5, 6, 2, 1, 4, 4, 2, 1, 2, 1, 5, 5]

tau, pvalue = vt.kendall_tau(x, y)
tau

0.030306509211290782

In [15]:
# PSI test
test_pd_psi['remaining_mat' + "_bin"] = pd.cut(test_pd_psi['remaining_mat'], bins=10, labels=False)
test_pd_psi['year_bins'] = np.where(test_pd_psi['vintage_year'] < 2007, 'period_1', 'period_2')

# Export data
test_pd_psi.to_csv('test_pd_psi.csv', index=False)

# Show results
kala = vt.population_stability_index(test_pd_psi, 'year_bins', 'remaining_mat_bin')
kala[1]

1.0344129494141174

In [16]:
# IV
zen = vt.information_value(german_data, 'checkingstatus', 'GoodCredit')

zen[1]

0.6660115033513336

In [17]:
zen[0]

,Variable,Value,All,Bad,Share,Bad Rate,Distribution Good,Distribution Bad,WoE,IV
0,checkingstatus,A11,274,135,0.274,0.492701,0.198571,0.450000,-0.818099,0.205693
1,checkingstatus,A12,269,105,0.269,0.390335,0.234286,0.350000,-0.401392,0.046447
3,checkingstatus,A13,63,14,0.063,0.222222,0.070000,0.046667,0.405465,0.009461
2,checkingstatus,A14,394,46,0.394,0.116751,0.497143,0.153333,1.176263,0.404410


In [18]:
# LGD t test
df = vt.lgd_t_test(lgd_t_test, 
                                'predicted_lgd', 
                                'realised_lgd',
                                level="segment",
                                segment_col='segment'
                               ).sort_values(by='segment')
df

,segment,N,realised_lgd_mean,pred_lgd_mean,s2,mean_error,t_stat,p_value
0,None,1000,0.49726,0.49357,0.00317,0.00369,2.072427,0.019241


In [19]:
df['p_value'].sum()

0.019240819246654106

In [20]:
# Migration matrix stats
vt.migration_matrices_statistics(pd_transition_matrix, 'period_1_ratings', 'period_2_ratings')

(0.18376068376068377, 0.1751824817518248)

In [21]:
# Bayesian error rate
vt.bayesian_error_rate(test_data_main, "default_flag", "predicted_pd")

0.106

In [22]:
# cier
vt.conditional_information_entropy_ratio(pd_calibration, "realised_pd", "count")

0.024548595310375846

In [23]:
# kullback_leibler_dist
vt.kullback_leibler_dist(pd_calibration, "realised_pd", "count")

0.006240325352140225

In [24]:
# test Gini
vt.gini(pd_test_data, "default_flag", "predicted_pd")

0.0017095099404838

In [25]:
# KS
vt.kolmogorov_smirnov_stat(test_data_main, "default_flag", "predicted_pd")

KstestResult(statistic=0.869, pvalue=0.0)

In [26]:
a = vt.kolmogorov_smirnov_stat(test_data_main, "default_flag", "predicted_pd")

In [27]:
a[0]

0.869

In [28]:
# Herfhindahl multiple period
# vt.herfindahl_multiple_period_test(pd_calibration, "rating", "realised_pd", "count")
buckets = ['A', 'B', 'C']
ratings1 = random.choices(buckets,  [0.4, 0.5, 0.1], k=1000)
test_data1 = pd.DataFrame({'ratings': ratings1})
ratings2 = random.choices(buckets,  [0.4, 0.5, 0.1], k=1000)
test_data2 = pd.DataFrame({'ratings': ratings2})

vt.herfindahl_multiple_period_test(test_data1, test_data2, "ratings")


,N_initial,h_initial,N_current,h_current,p_value,reject
B,516,None,494,None,None,None
A,398,None,406,None,None,None
C,86,None,100,None,None,None
total,1000,0.432056,1000,0.418872,0.548078,False


In [29]:
# Loss Capture Ratio
vt.loss_capture_ratio(lgd_t_test['ead'], lgd_t_test['predicted_lgd'], lgd_t_test['realised_lgd'])

1.0000874459653837

In [30]:
lgd_t_test.head()

,facility,predicted_lgd,realised_lgd,segment,ead
0,1,0.80,0.83,8,5860
1,2,0.75,0.81,7,6653
2,3,0.42,0.45,4,3192
3,4,0.99,1.00,9,7322
4,5,0.90,0.86,9,5513


In [31]:
# CLAR
vt.cumulative_lgd_accuracy_ratio(lgd_dataset, 'predicted_outcome', 'realised_outcome')

3.1999999999999997

In [32]:
# Migration Matrix Stability
vt.migration_matrix_stability(pd_transition_matrix, 'period_1_ratings', 'period_2_ratings')

C:\projects\meliora\src\meliora\core.py:1395: RuntimeWarning: invalid value encountered in double_scalars
  z_ij = num / np.sqrt(den_a + den_b + den_c)
C:\projects\meliora\src\meliora\core.py:1385: RuntimeWarning: invalid value encountered in double_scalars
  z_ij = num / np.sqrt(den_a + den_b + den_c)


(period_2_ratings         1         2         3         4         5         6  \
 period_1_ratings                                                               
 1                      NaN  5.441072 -1.315587  3.585686       NaN       NaN   
 2                -1.465656       NaN  0.301985 -0.301985  2.750494       NaN   
 3                -2.466006  0.000000       NaN  0.000000  0.635724  2.111195   
 4                 0.378968  0.918559 -0.580948       NaN  0.334021  0.000000   
 5                 2.905370  0.501531 -1.420463 -0.378625       NaN -2.493813   
 6                      NaN  3.877284 -0.448308 -1.799123  1.290339       NaN   
 7                      NaN       NaN  3.119251 -1.978141  2.689264 -1.538968   
 8                      NaN       NaN       NaN  1.816590  1.562315 -1.562315   
 
 period_2_ratings         7         8  
 period_1_ratings                      
 1                      NaN       NaN  
 2                      NaN       NaN  
 3                      NaN 

In [33]:
q = vt.migration_matrix_stability(pd_transition_matrix, 'period_1_ratings', 'period_2_ratings')

C:\projects\meliora\src\meliora\core.py:1395: RuntimeWarning: invalid value encountered in double_scalars
  z_ij = num / np.sqrt(den_a + den_b + den_c)
C:\projects\meliora\src\meliora\core.py:1385: RuntimeWarning: invalid value encountered in double_scalars
  z_ij = num / np.sqrt(den_a + den_b + den_c)


In [34]:
q[0].sum().sum()

23.81875738112634

In [35]:
q[1].sum().sum()

23.11530838816691

In [36]:
# Pearson correlation
x = [1,2,3,4,5]
y = [5,6,7,8,7]
vt.pearson_correlation(x, y)

SpearmanrResult(correlation=0.8207826816681233, pvalue=0.08858700531354381)

In [37]:
# ELBE t test
vt.elbe_t_test(lgd_t_test, "predicted_lgd", "realised_lgd")

,facilities,lgd_mean,t_stat,p_value
0,1000,0.49357,2.072427,0.038482


In [38]:
# Loss Shortfall
vt.loss_shortfall(lgd_t_test, "ead", "predicted_lgd", "realised_lgd")

-0.008480989922580617

In [39]:
# Mean Absolute Deviation
vt.mean_absolute_deviation(pd_calibration, "rating", "realised_pd", "count")

6999.929781818181

In [40]:
# Normal test
vt.normal_test(pd_calibration['predicted_pd'], pd_calibration['realised_pd'])

,estimate,t_stat,p_value,outcome
0,0.001,0.015093,0.505873,NaN


In [41]:
q = vt.normal_test(pd_calibration['predicted_pd'], pd_calibration['realised_pd'])


#### Relevant links
- https://machinelearningmastery.com/divergence-between-probability-distributions/
- https://medium.com/@monadsblog/the-kullback-leibler-divergence-5071c707a4a6
- https://documentation.sas.com/doc/en/pgmsascdc/v_011/statug/statug_code_logiex19.htm